In [1]:
 #!pip install numpy pandas tqdm scikit-learn segmentation-models tensorflow-addons 

In [2]:
#!pip install albumentations --user

In [1]:
import tensorflow as tf
import time
print(tf.__version__)
print('GPU Available: ', tf.test.is_gpu_available())


# Forzar una operación para comprobar el uso de la GPU
with tf.device('/GPU:0'):
    a = tf.random.normal([10000, 10000])
    b = tf.random.normal([10000, 10000])
    start_time = time.time()
    c = tf.matmul(a, b)
    end_time = time.time()
    print('GPU Available: ', tf.test.is_gpu_available())


print(f"Tiempo de ejecución en la GPU: {end_time - start_time} segundos")
print("GPUs disponibles: ", tf.config.list_physical_devices('GPU'))




2024-10-10 17:05:14.985167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 17:05:15.038312: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 17:05:15.054587: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-10 17:05:15.420483: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

2.10.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Available:  False


2024-10-10 17:05:15.856481: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 17:05:15.871543: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-10 17:05:15.875931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/targets/x86_64-linux/lib:
2024-10-10 17:05:15.875941: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the mi

GPU Available:  False
Tiempo de ejecución en la GPU: 1.271162748336792 segundos
GPUs disponibles:  []


2024-10-10 17:05:17.440227: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-10-10 17:05:17.441105: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
import tensorflow as tf
import os
import sys
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import numpy as np
import pandas as pd
import random
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score

module_path = "/home/voicelab/Desktop/segmentation_glottis/UNet/drive/colab_glottis/colab_glottis/GlottisNetV2"
if module_path not in sys.path:
    sys.path.append(module_path)
#Drive
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.DataGenerator import DataGenerator
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.data import load_data, metric_mape, mape_ap, mape_pp
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Utils.Callbacks import get_callbacks
#from drive.MyDrive.Glottis.colab_glottis.GlottisNetV2.Models.GlottisNetV2_e import glottisnetV2_e

from Models.GlottisNetV2_e import glottisnetV2_e
from Utils.DataGenerator_onlySeg import DataGenerator
from Utils.data import load_data, metric_mape, mape_ap, mape_pp
from Utils.Callbacks import get_callbacks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Segmentation Models: using `tf.keras` framework.


/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [5]:
import pandas as pd
import os

from tqdm import tqdm

def is_image_valid(image_path):
    return os.path.getsize(image_path) > 0

img_training = "/home/voicelab/Desktop/segmentation_glottis/BAGLS_ROI/training" # TODO

N_train = 55750

cols = ['z', 'path']
df_imgs_train = pd.DataFrame(columns=cols)
df_segs_train = pd.DataFrame(columns=cols)

for i in tqdm(range(0, N_train )):  # Asegúrate de que el rango esté correctamente definido
    rgb_path = os.path.join(img_training, f"{i}.png")
    mask_path = os.path.join(img_training, f"{i}_seg.png")

    # Solo agrega las rutas si tanto la imagen como la máscara son válidas
    if os.path.exists(rgb_path) and os.path.exists(mask_path) and is_image_valid(rgb_path) and is_image_valid(mask_path):
        row_imgs = {'z': [i], 'path': [rgb_path]}
        row_segs = {'z': [i], 'path': [mask_path]}

        df_imgs_train = pd.concat([df_imgs_train, pd.DataFrame(row_imgs)])
        df_segs_train = pd.concat([df_segs_train, pd.DataFrame(row_segs)])
    else:
        print(f"Omitiendo debido a imagen/máscara inválida o inexistente: {rgb_path}, {mask_path}")

print(df_imgs_train.tail())
print(df_segs_train.tail())

print('Created IDs for training images.')


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55750/55750 [00:48<00:00, 1145.08it/s]

       z                                               path
0  55745  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55746  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55747  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55748  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55749  /home/voicelab/Desktop/segmentation_glottis/BA...
       z                                               path
0  55745  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55746  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55747  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55748  /home/voicelab/Desktop/segmentation_glottis/BA...
0  55749  /home/voicelab/Desktop/segmentation_glottis/BA...
Created IDs for training images.


In [6]:
from tensorflow.keras.models import load_model

'''Training'''
# Set random seed for reproducible training
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
rand=np.random.seed(SEED)
random.seed(SEED)
tf.compat.v1.set_random_seed(SEED)

# Set parameters
BATCH_SIZE = 8
FILTERS = 16
LAYERS= 4
LEARNING_RATE = 0.2e-3
EPOCHS = 30
TARGET_HEIGHT = 128
TARGET_WIDTH = 256
SHUFFLE = True
AUGMENT = False
MODEL_PATH = r"/home/voicelab/Desktop/segmentation_glottis/UNet/model/model128.h5" #TODO
STEPS_PATH = r"/home/voicelab/Desktop/segmentation_glottis/UNet/model/steps128" #TODO
N_STEPS = 5 # Save every #N_STEPS epoch
RADIUS = 15

model = glottisnetV2_e(input_size=(TARGET_HEIGHT, TARGET_WIDTH, 1), layers=LAYERS, filters=FILTERS)

# Hard split of training and validation data
train_imgs, val_imgs, train_segs, val_segs = train_test_split(df_imgs_train,
                                                              df_segs_train,
                                                              test_size = 0.1,
                                                              random_state = SEED)


# Training data --> Augmentation and Shuffle
#training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   #target_width = TARGET_WIDTH, shuffle = SHUFFLE, df_coordinates = training_data, \
                                   #augment = AUGMENT, radius=RADIUS)
training_generator = DataGenerator(train_imgs, train_segs, batch_size = BATCH_SIZE, target_height = TARGET_HEIGHT, \
                                   target_width = TARGET_WIDTH, shuffle = SHUFFLE, augment = AUGMENT, radius=RADIUS)
# Validation data
validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                    augment = False, radius=RADIUS)
#validation_generator = DataGenerator(val_imgs, val_segs, target_height = TARGET_HEIGHT, \
                                     #target_width = TARGET_WIDTH, batch_size = BATCH_SIZE, shuffle = False, \
                                     #df_coordinates = training_data, augment = False, radius=RADIUS)
# Compile model with dice_loss for segmentation, mse for prediction maps and use Adam as optimizer
# First exit: predictions of anterior and posterior points (2 channels)
# Second exit: Segmentations

for layer in model.layers:
    if 'decode_ap' in layer.name or 'ap.pred' in layer.name:
        layer.trainable = False



model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              metrics={'seg': ['acc', iou_score]},
              loss={'seg': dice_loss})


# Train model on dataset and save it
model.fit(training_generator, validation_data= validation_generator, epochs = EPOCHS,
                    callbacks = get_callbacks(MODEL_PATH, model, N_STEPS, STEPS_PATH))


Epoch 1/30


2024-09-24 22:53:06.587795: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


2024-09-24 22:53:09.763365: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-09-24 22:53:09.806859: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-09-24 22:53:10.812773: I external/local_xla/xla/service/service.cc:168] XLA service 0x7d7e633cd600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-24 22:53:10.812789: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4080 SUPER, Compute Capability 8.9
2024-09-24 22:53:10.815426: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1727229190.866487  922362 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


6271/6271 [==============================] - 112s 17ms/step - loss: 0.5776 - seg_loss: 0.5776 - seg_acc: 0.9047 - seg_iou_score: 0.2733 - val_loss: 0.5314 - val_seg_loss: 0.5314 - val_seg_acc: 0.9249 - val_seg_iou_score: 0.3105
Epoch 2/30
   5/6271 [..............................] - ETA: 1:25 - loss: 0.4785 - seg_loss: 0.4785 - seg_acc: 0.9279 - seg_iou_score: 0.3584

/home/voicelab/miniconda3/envs/tmpenv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


6271/6271 [==============================] - 101s 16ms/step - loss: 0.5359 - seg_loss: 0.5359 - seg_acc: 0.9214 - seg_iou_score: 0.3064 - val_loss: 0.5245 - val_seg_loss: 0.5245 - val_seg_acc: 0.9235 - val_seg_iou_score: 0.3164
Epoch 3/30
6271/6271 [==============================] - 98s 16ms/step - loss: 0.5345 - seg_loss: 0.5345 - seg_acc: 0.9219 - seg_iou_score: 0.3076 - val_loss: 0.5334 - val_seg_loss: 0.5334 - val_seg_acc: 0.9236 - val_seg_iou_score: 0.3086
Epoch 4/30
6271/6271 [==============================] - 101s 16ms/step - loss: 0.5337 - seg_loss: 0.5337 - seg_acc: 0.9223 - seg_iou_score: 0.3082 - val_loss: 0.5323 - val_seg_loss: 0.5323 - val_seg_acc: 0.9242 - val_seg_iou_score: 0.3096
Epoch 5/30
6271/6271 [==============================] - 104s 17ms/step - loss: 0.5337 - seg_loss: 0.5337 - seg_acc: 0.9222 - seg_iou_score: 0.3084 - val_loss: 0.5345 - val_seg_loss: 0.5345 - val_seg_acc: 0.9233 - val_seg_iou_score: 0.3078
Epoch 6/30
6271/6271 [==============================] - 

In [3]:
import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    with tf.device('/GPU:0'):
        # Tu código para probar la GPU aquí
        a = tf.random.normal([100, 100])
        b = tf.random.normal([100, 100])
        c = tf.matmul(a, b)
        print(c)
else:
    print("GPU no disponible")


GPU no disponible
